In [2]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

# ---------- 1. Read Middlebury .flo file ----------
def read_flo_file(filename):
    """Read .flo optical flow file in Middlebury format"""
    with open(filename, 'rb') as f:
        magic = np.fromfile(f, np.float32, count=1)[0]
        if magic != 202021.25:
            raise Exception("Magic number incorrect. Invalid .flo file")
        w = np.fromfile(f, np.int32, count=1)[0]
        h = np.fromfile(f, np.int32, count=1)[0]
        data = np.fromfile(f, np.float32, count=2*w*h)
        flow = np.resize(data, (h, w, 2))
        return flow

# ---------- 2. Convert optical flow to RGB visualization ----------
def flow_to_rgb(flow):
    """Convert flow field to RGB for visualization"""
    h, w = flow.shape[:2]
    fx = flow[..., 0]
    fy = flow[..., 1]

    # Compute magnitude and angle
    mag, ang = cv2.cartToPolar(fx, fy)

    # HSV map
    hsv = np.zeros((h, w, 3), dtype=np.uint8)
    hsv[..., 0] = (ang * 180 / np.pi / 2)      # Hue
    hsv[..., 1] = 255                          # Saturation
    hsv[..., 2] = np.clip(mag * 10, 0, 255)    # Value (scaled mag)

    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return rgb

# ---------- 3. Main convert ----------
input_flo = "/home/serverai/ltdoanh/AniUnFlow/data/AnimeRun_v2/train/Flow/agent_basement2_weapon_approach/forward/Image0186.flo"
output_dir = "/home/serverai/ltdoanh/AniUnFlow/outputs/flow_png/"
os.makedirs(output_dir, exist_ok=True)

# Read flow
flow = read_flo_file(input_flo)
print("Flow shape:", flow.shape)   # Expect (H, W, 2)

# Generate RGB visualization
flow_rgb = flow_to_rgb(flow)

# Save visualization
out_png = os.path.join(output_dir, "Image0186_flow_vis.png")
cv2.imwrite(out_png, flow_rgb)

# Save U/V debug image
u = flow[..., 0]
v = flow[..., 1]

u_img = cv2.normalize(u, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
v_img = cv2.normalize(v, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
uv = np.stack([u_img, v_img, np.zeros_like(u_img)], axis=2)

out_uv = os.path.join(output_dir, "Image0186_flow_uv.png")
cv2.imwrite(out_uv, uv)

print("Saved:", out_png)
print("Saved:", out_uv)


Flow shape: (429, 1024, 2)
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/flow_png/Image0186_flow_vis.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/flow_png/Image0186_flow_uv.png


In [4]:
import os
import glob
import numpy as np
import cv2

# ---------- 1. Read Middlebury .flo file ----------
def read_flo_file(filename):
    """Read .flo optical flow file in Middlebury format"""
    with open(filename, 'rb') as f:
        magic = np.fromfile(f, np.float32, count=1)[0]
        if magic != 202021.25:
            raise Exception(f"[ERROR] {filename}: Invalid .flo file (wrong magic number)")
        w = np.fromfile(f, np.int32, count=1)[0]
        h = np.fromfile(f, np.int32, count=1)[0]
        data = np.fromfile(f, np.float32, count=2*w*h)
        flow = np.resize(data, (h, w, 2))
        return flow

# ---------- 2. Convert optical flow to RGB visualization ----------
def flow_to_rgb(flow):
    """Convert flow field to RGB (HSV -> BGR) for visualization"""
    h, w = flow.shape[:2]
    fx = flow[..., 0]
    fy = flow[..., 1]

    mag, ang = cv2.cartToPolar(fx, fy)

    hsv = np.zeros((h, w, 3), dtype=np.uint8)
    hsv[..., 0] = (ang * 180 / np.pi / 2)      # Hue
    hsv[..., 1] = 255                          # Saturation
    hsv[..., 2] = np.clip(mag * 10, 0, 255)    # Value (scaled mag)

    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return rgb

# ---------- 3. Batch convert folder ----------
flo_dir = "/home/serverai/ltdoanh/AniUnFlow/data/AnimeRun_v2/train/Flow/agent_indoor4_dodges/forward"
out_dir = os.path.join("/home/serverai/ltdoanh/AniUnFlow/outputs", "forward_vis")  # thư mục output cùng cấp
os.makedirs(out_dir, exist_ok=True)

flo_files = sorted(glob.glob(os.path.join(flo_dir, "*.flo")))
print(f"Found {len(flo_files)} .flo files")

for flo_path in flo_files:
    try:
        flow = read_flo_file(flo_path)
    except Exception as e:
        print(e)
        continue

    flow_rgb = flow_to_rgb(flow)

    base = os.path.splitext(os.path.basename(flo_path))[0]  # Image0186
    out_png = os.path.join(out_dir, base + "_flow_vis.png")
    cv2.imwrite(out_png, flow_rgb)

    print("Saved:", out_png)

print("Done.")


Found 44 .flo files
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/forward_vis/Image0182_flow_vis.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/forward_vis/Image0183_flow_vis.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/forward_vis/Image0184_flow_vis.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/forward_vis/Image0185_flow_vis.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/forward_vis/Image0186_flow_vis.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/forward_vis/Image0187_flow_vis.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/forward_vis/Image0188_flow_vis.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/forward_vis/Image0189_flow_vis.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/forward_vis/Image0190_flow_vis.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/forward_vis/Image0191_flow_vis.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/forward_vis/Image0192_flow_vis.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/forward_vis/Image01

In [1]:
import os
import glob
import numpy as np
import cv2
import sys
sys.path.append("/home/serverai/ltdoanh/AniUnFlow")
from utils.flow_viz import flow_to_image, compute_flow_magnitude_radmax

# ---------- 1. Read Middlebury .flo file ----------
def read_flo_file(filename):
    """Read .flo optical flow file in Middlebury format."""
    with open(filename, 'rb') as f:
        magic = np.fromfile(f, np.float32, count=1)[0]
        if magic != 202021.25:
            raise Exception(f"[ERROR] {filename}: Invalid .flo file (wrong magic number)")
        w = np.fromfile(f, np.int32, count=1)[0]
        h = np.fromfile(f, np.int32, count=1)[0]
        data = np.fromfile(f, np.float32, count=2 * w * h)
        flow = np.resize(data, (h, w, 2))
        return flow


# ---------- 2. Paths ----------
# /home/serverai/ltdoanh/AniUnFlow/data/AnimeRun_v2/train/Frame_Anime/cami_06_03_A(renew_background)/color_1
flo_dir = "/home/serverai/ltdoanh/AniUnFlow/data/AnimeRun_v2/train/Flow/cami_06_03_A(renew_background)/forward"
out_dir = os.path.join("/home/serverai/ltdoanh/AniUnFlow/outputs/aft_eval", "forward_vis_tomrunia")   # output folder in the same directory
os.makedirs(out_dir, exist_ok=True)

flo_files = sorted(glob.glob(os.path.join(flo_dir, "*.flo")))
print(f"Found {len(flo_files)} .flo files in {flo_dir}")

if not flo_files:
    raise SystemExit("No .flo files found, please check the path.")

# ---------- 3. First pass: read all flows to compute shared rad_max ----------
flows = []
for fp in flo_files:
    flow = read_flo_file(fp)
    flows.append(flow)

rad_max = compute_flow_magnitude_radmax(flows, robust_percentile=95)
print(f"Shared rad_max (p95) = {rad_max:.4f}")

# ---------- 4. Second pass: convert each flow to color image and save ----------
for flo_path, flow in zip(flo_files, flows):
    # Use fixed rad_max for consistent coloring across the whole sequence
    img = flow_to_image(
        flow,
        clip_flow=None,           # or e.g. 50.0 if you want to hard-clip
        robust_percentile=None,   # disable internal percentile, we already computed rad_max
        convert_to_bgr=True,      # True because OpenCV expects BGR
        rad_max=rad_max
    )

    base = os.path.splitext(os.path.basename(flo_path))[0]  # e.g. Image0186
    out_png = os.path.join(out_dir, base + "_flow.png")
    cv2.imwrite(out_png, img)
    print("Saved:", out_png)

print("Done.")


Found 64 .flo files in /home/serverai/ltdoanh/AniUnFlow/data/AnimeRun_v2/train/Flow/cami_06_03_A(renew_background)/forward
Shared rad_max (p95) = 45.0709
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/aft_eval/forward_vis_tomrunia/0110_flow.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/aft_eval/forward_vis_tomrunia/0111_flow.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/aft_eval/forward_vis_tomrunia/0112_flow.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/aft_eval/forward_vis_tomrunia/0113_flow.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/aft_eval/forward_vis_tomrunia/0114_flow.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/aft_eval/forward_vis_tomrunia/0115_flow.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/aft_eval/forward_vis_tomrunia/0116_flow.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/aft_eval/forward_vis_tomrunia/0117_flow.png
Saved: /home/serverai/ltdoanh/AniUnFlow/outputs/aft_eval/forward_vis_tomrunia/0118_flow.png
Saved: /home/serve